In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import torch
random.seed(666)
torch.manual_seed(666)

In [3]:
from pathlib import Path
OUTPUT_DIR = Path("output/baseline")

from utils import safe_clear_output_dir
safe_clear_output_dir(OUTPUT_DIR)

Cleared 0 files from output\baseline


# Load Data

In [ ]:

from npz_loader import build_episodes_from_sequences
from npz_loader import discover_game_npz_paths, get_sequences_by_game, fix_obs_paths, build_episodes_from_sequences

holdout_games = [
    Path(r"dataset\BeamRiderNoFrameskip-v4\BeamRiderNoFrameskip-v4"),
    Path(r"dataset\BreakoutNoFrameskip-v4\BreakoutNoFrameskip-v4"),
]

all_game_dirs = [
    Path(r"dataset\EnduroNoFrameskip-v4\EnduroNoFrameskip-v4"),
    Path(r"dataset\MsPacmanNoFrameskip-v4\MsPacmanNoFrameskip-v4"),
    Path(r"dataset\PongNoFrameskip-v4\PongNoFrameskip-v4"),
    # Path(r"dataset\QbertNoFrameskip-v4\QbertNoFrameskip-v4"),
    # Path(r"dataset\SeaquestNoFrameskip-v4\SeaquestNoFrameskip-v4"),
    # Path(r"dataset\SpaceInvadersNoFrameskip-v4\SpaceInvadersNoFrameskip-v4"),
]
all_game_dirs.extend(holdout_games)

npz_paths_by_game = discover_game_npz_paths(all_game_dirs)
game_to_sequences = get_sequences_by_game(npz_paths_by_game)
sequences_by_game = fix_obs_paths(game_to_sequences, dataset_root="dataset")
episodes = build_episodes_from_sequences(sequences_by_game)

TypeError: 'NoneType' object is not iterable

In [ ]:
from utils import sample_list
sampled_episodes = sample_list(episodes, dataset_fraction=0.1)

Sampled 108 episodes (10.0% of 1080 total)


In [ ]:
from episode_dataset import make_train_val_dataloaders

holdout_game_names = [game.name for game in holdout_games]

# This must be called for all data the model will see
# Otherwise the binning may not be correct for the new data
main_bundle, holdout_bundle, bins = make_train_val_dataloaders(
    episodes=sampled_episodes,
    holdout_games=holdout_games,
    train_frac=0.9,
    timestep_window_size=4,
    image_size=(84, 84),
    batch_size=32,
)

# Dataloader - Main Games

In [ ]:
from dataloader_stats import inspect_dataloader

inspect_dataloader(main_bundle.train_loader, "Main Train", output_dir=OUTPUT_DIR)
inspect_dataloader(main_bundle.val_loader, "Main Validate", output_dir=OUTPUT_DIR)

=== Dataloader Sanity Check: Main Train ===
Used batches: 50
First batch shapes: {'frames': (32, 4, 3, 84, 84), 'actions': (32, 4), 'rewards': (32, 4), 'rtg': (32, 4)}
Total samples (timesteps): 6400
Games in subset: 3 -> {'EnduroNoFrameskip-v4': 1234, 'MsPacmanNoFrameskip-v4': 328, 'BreakoutNoFrameskip-v4': 38}
Actions: min=0, max=8, unique=9
Rewards: min=-1.000, max=1.000
RTG: min=-13.000, max=185.000
Reward bins: unique=[0 1 2]
RTG bins: min=1, max=199
NaNs in rewards? False
NaNs in RTG? False
Saved plot to output\baseline\Main_Train.png
{'first_shapes': {'frames': (32, 4, 3, 84, 84), 'actions': (32, 4), 'rewards': (32, 4), 'rtg': (32, 4)}, 'n_batches_used': 50, 'n_timesteps': 6400, 'game_counts': {'EnduroNoFrameskip-v4': 1234, 'MsPacmanNoFrameskip-v4': 328, 'BreakoutNoFrameskip-v4': 38}, 'actions_min': 0.0, 'actions_max': 8.0, 'n_unique_actions': 9, 'rewards_min': -1.0, 'rewards_max': 1.0, 'rtg_min': -13.0, 'rtg_max': 185.0, 'reward_bins_unique': array([0, 1, 2]), 'rtg_bins_min': 1

{'first_shapes': {'frames': (32, 4, 3, 84, 84),
  'actions': (32, 4),
  'rewards': (32, 4),
  'rtg': (32, 4)},
 'n_batches_used': 50,
 'n_timesteps': 6400,
 'game_counts': {'MsPacmanNoFrameskip-v4': 806, 'EnduroNoFrameskip-v4': 794},
 'actions_min': 0.0,
 'actions_max': 8.0,
 'n_unique_actions': 9,
 'rewards_min': -1.0,
 'rewards_max': 1.0,
 'rtg_min': 0.0,
 'rtg_max': 184.0,
 'reward_bins_unique': array([0, 1, 2]),
 'rtg_bins_min': 14,
 'rtg_bins_max': 198}

# Dataloader - Holdout Games

In [ ]:
inspect_dataloader(holdout_bundle.train_loader, "Holdout Train", output_dir=OUTPUT_DIR)
inspect_dataloader(holdout_bundle.val_loader, "Holdout Validate", output_dir=OUTPUT_DIR)

=== Dataloader Sanity Check: Holdout Train ===
Used batches: 50
First batch shapes: {'frames': (32, 4, 3, 84, 84), 'actions': (32, 4), 'rewards': (32, 4), 'rtg': (32, 4)}
Total samples (timesteps): 6400
Games in subset: 1 -> {'BeamRiderNoFrameskip-v4': 1600}
Actions: min=0, max=8, unique=9
Rewards: min=0.000, max=1.000
RTG: min=0.000, max=15.000
Reward bins: unique=[1 2]
RTG bins: min=14, max=29
NaNs in rewards? False
NaNs in RTG? False
Saved plot to output\baseline\Holdout_Train.png
{'first_shapes': {'frames': (32, 4, 3, 84, 84), 'actions': (32, 4), 'rewards': (32, 4), 'rtg': (32, 4)}, 'n_batches_used': 50, 'n_timesteps': 6400, 'game_counts': {'BeamRiderNoFrameskip-v4': 1600}, 'actions_min': 0.0, 'actions_max': 8.0, 'n_unique_actions': 9, 'rewards_min': 0.0, 'rewards_max': 1.0, 'rtg_min': 0.0, 'rtg_max': 15.0, 'reward_bins_unique': array([1, 2]), 'rtg_bins_min': 14, 'rtg_bins_max': 29}
=== Dataloader Sanity Check: Holdout Validate ===
Used batches: 45
First batch shapes: {'frames': (3

{'first_shapes': {'frames': (32, 4, 3, 84, 84),
  'actions': (32, 4),
  'rewards': (32, 4),
  'rtg': (32, 4)},
 'n_batches_used': 45,
 'n_timesteps': 5720,
 'game_counts': {'BeamRiderNoFrameskip-v4': 1430},
 'actions_min': 0.0,
 'actions_max': 8.0,
 'n_unique_actions': 9,
 'rewards_min': 0.0,
 'rewards_max': 1.0,
 'rtg_min': 0.0,
 'rtg_max': 15.0,
 'reward_bins_unique': array([1, 2]),
 'rtg_bins_min': 14,
 'rtg_bins_max': 29}

# Train - Main Games

In [ ]:
from mgdt_model_trainer import Encoder, train_mgdt

model, main_train_stats, main_val_stats = train_mgdt(
    bins=bins,
    dataloader_train=main_bundle.train_loader,
    dataloader_val=main_bundle.val_loader,
    encoder_type=Encoder.Patch,
    num_epochs=2,
)

Epoch 2/2: 100%|██████████| 2703/2703 [02:21<00:00, 19.08it/s]


In [ ]:
from mgdt_model_stats import plot_losses
plot_losses(main_train_stats, main_val_stats, output_dir=OUTPUT_DIR, title_prefix="Main")

Saved plot to output\baseline\Main_losses_per_head.png
Saved plot to output\baseline\Main_losses_combined.png
Saved plot to output\baseline\Main_losses_ema_per_head.png
Saved plot to output\baseline\Main_losses_ema_combined.png


# Train - Holdout Games

In [ ]:
from mgdt_model_trainer import Encoder, train_mgdt

model, holdout_train_stats, holdout_val_stats = train_mgdt(
    model=model,
    bins=bins,
    dataloader_train=holdout_bundle.train_loader,
    dataloader_val=holdout_bundle.val_loader,
    encoder_type=Encoder.Patch,
    num_epochs=1,
)

Finetune 1/1: 100%|██████████| 378/378 [00:41<00:00,  9.03it/s]


In [ ]:
from mgdt_model_stats import plot_losses
plot_losses(holdout_train_stats, holdout_val_stats, output_dir=OUTPUT_DIR, title_prefix="Holdout")

Saved plot to output\baseline\Holdout_losses_per_head.png
Saved plot to output\baseline\Holdout_losses_combined.png
Saved plot to output\baseline\Holdout_losses_ema_per_head.png
Saved plot to output\baseline\Holdout_losses_ema_combined.png


# Comparison

In [ ]:
from mgdt_model_stats import plot_holdout_comparison

plot_holdout_comparison(main_train_stats, main_val_stats, holdout_train_stats, holdout_val_stats, output_dir=OUTPUT_DIR, title_prefix="Baseline")

Saved plot to output\baseline\Baseline_main_vs_holdout.png
Saved plot to output\baseline\Baseline_per_head_loss.png
Saved plot to output\baseline\Baseline_accuracy.png

HOLDOUT ADAPTATION SUMMARY
Main training steps: 5406
Holdout fine-tune steps: 378

Main training - Final loss (EMA): 1.8595
Holdout fine-tune - Initial loss (EMA): 3.2187
Holdout fine-tune - Final loss (EMA): 1.7087
Holdout loss reduction: 1.5100

Holdout reached main's final loss at step 103 (27.2% of fine-tuning)


# Model Info and Save

In [ ]:
total_params = sum(param.numel() for param in model.parameters())
print(f"Total parameters: {total_params/1e6:.0f}M")

Total parameters: 20M


In [ ]:
from utils import save_checkpoint

save_checkpoint(model, main_train_stats, main_val_stats, output_dir=OUTPUT_DIR)

Model and stats saved to output\baseline\model.pt


WindowsPath('output/baseline/model.pt')